# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Genetic-Analysis-of-a-Metazoan-Pathway-using-Transcriptomic-Phenotypes-Supplementary-and-Extended-Material" data-toc-modified-id="Genetic-Analysis-of-a-Metazoan-Pathway-using-Transcriptomic-Phenotypes-Supplementary-and-Extended-Material-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Genetic Analysis of a Metazoan Pathway using Transcriptomic Phenotypes Supplementary and Extended Material</a></div><div class="lev1 toc-item"><a href="#Folder-Structure" data-toc-modified-id="Folder-Structure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Folder Structure</a></div><div class="lev1 toc-item"><a href="#Read-alignment-and-differential-expression-analysis" data-toc-modified-id="Read-alignment-and-differential-expression-analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Read alignment and differential expression analysis</a></div><div class="lev1 toc-item"><a href="#Introduction:-Genetic-Analysis-Using-Global-Expression-Measurements" data-toc-modified-id="Introduction:-Genetic-Analysis-Using-Global-Expression-Measurements-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Introduction: Genetic Analysis Using Global Expression Measurements</a></div><div class="lev2 toc-item"><a href="#PCA-Analysis-of-TPM-Data" data-toc-modified-id="PCA-Analysis-of-TPM-Data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>PCA Analysis of TPM Data</a></div><div class="lev1 toc-item"><a href="#Figure-1.-Dendrogram-Clustering" data-toc-modified-id="Figure-1.-Dendrogram-Clustering-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Figure 1. Dendrogram Clustering</a></div><div class="lev2 toc-item"><a href="#Filtering-Does-Not-Seriously-Alter-Differentially-Expressed-Genes" data-toc-modified-id="Filtering-Does-Not-Seriously-Alter-Differentially-Expressed-Genes-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Filtering Does Not Seriously Alter Differentially Expressed Genes</a></div><div class="lev1 toc-item"><a href="#Bayesian-versus-Spearman-Regressions" data-toc-modified-id="Bayesian-versus-Spearman-Regressions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Bayesian versus Spearman Regressions</a></div><div class="lev2 toc-item"><a href="#Spearman-Regression-Method" data-toc-modified-id="Spearman-Regression-Method-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Spearman Regression Method</a></div><div class="lev2 toc-item"><a href="#Bayesian-Robust-Regression" data-toc-modified-id="Bayesian-Robust-Regression-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Bayesian Robust Regression</a></div><div class="lev2 toc-item"><a href="#Figure-4.-Complex-Regulation-Generates-Detectable-Patterns-in-Transcriptomes" data-toc-modified-id="Figure-4.-Complex-Regulation-Generates-Detectable-Patterns-in-Transcriptomes-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Figure 4. Complex Regulation Generates Detectable Patterns in Transcriptomes</a></div><div class="lev2 toc-item"><a href="#Figure-2.-Positive-Regulatory-Relationships-Can-Be-Identified-By-Transcriptomic-Correlation" data-toc-modified-id="Figure-2.-Positive-Regulatory-Relationships-Can-Be-Identified-By-Transcriptomic-Correlation-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Figure 2. Positive Regulatory Relationships Can Be Identified By Transcriptomic Correlation</a></div><div class="lev1 toc-item"><a href="#Pairwise-Analysis-of-All-Genes-Using-Spearman-Correlation" data-toc-modified-id="Pairwise-Analysis-of-All-Genes-Using-Spearman-Correlation-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Pairwise Analysis of All Genes Using Spearman Correlation</a></div><div class="lev1 toc-item"><a href="#Hypergeometric-Analysis:" data-toc-modified-id="Hypergeometric-Analysis:-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Hypergeometric Analysis:</a></div><div class="lev1 toc-item"><a href="#Analysis-Using-Robust-Regression" data-toc-modified-id="Analysis-Using-Robust-Regression-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Analysis Using Robust Regression</a></div><div class="lev2 toc-item"><a href="#Figure-3.-Pairwise-regression-values-between-all-single-mutants" data-toc-modified-id="Figure-3.-Pairwise-regression-values-between-all-single-mutants-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Figure 3. Pairwise regression values between all single mutants</a></div><div class="lev1 toc-item"><a href="#Extracting-functional-relationships-between-genes" data-toc-modified-id="Extracting-functional-relationships-between-genes-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Extracting functional relationships between genes</a></div><div class="lev2 toc-item"><a href="#Figure-4.-Weighted-Correlations-Reflect-Functional-Distance" data-toc-modified-id="Figure-4.-Weighted-Correlations-Reflect-Functional-Distance-10.1"><span class="toc-item-num">10.1&nbsp;&nbsp;</span>Figure 4. Weighted Correlations Reflect Functional Distance</a></div><div class="lev1 toc-item"><a href="#Double-Mutant-Analysis" data-toc-modified-id="Double-Mutant-Analysis-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Double Mutant Analysis</a></div><div class="lev1 toc-item"><a href="#Defining-the-Hypoxia-Response" data-toc-modified-id="Defining-the-Hypoxia-Response-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Defining the Hypoxia Response</a></div><div class="lev1 toc-item"><a href="#The-genes-upregulated-by-egl-9-should-be-down-in-hif-1." data-toc-modified-id="The-genes-upregulated-by-egl-9-should-be-down-in-hif-1.-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>The genes upregulated by <em>egl-9</em> should be down in <em>hif-1</em>.</a></div><div class="lev1 toc-item"><a href="#Quality-Control" data-toc-modified-id="Quality-Control-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Quality Control</a></div><div class="lev1 toc-item"><a href="#Correlation-Graph" data-toc-modified-id="Correlation-Graph-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Correlation Graph</a></div><div class="lev1 toc-item"><a href="#An-in-silico-qPCR-experiment:" data-toc-modified-id="An-in-silico-qPCR-experiment:-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>An <em>in silico</em> qPCR experiment:</a></div><div class="lev1 toc-item"><a href="#Define-the-Absolute-Gold-Set" data-toc-modified-id="Define-the-Absolute-Gold-Set-17"><span class="toc-item-num">17&nbsp;&nbsp;</span>Define the Absolute Gold Set</a></div><div class="lev1 toc-item"><a href="#Hypoxia-Positive-Interaction" data-toc-modified-id="Hypoxia-Positive-Interaction-18"><span class="toc-item-num">18&nbsp;&nbsp;</span>Hypoxia Positive Interaction</a></div><div class="lev1 toc-item"><a href="#Searching-for-a-TF-that-is-activated-by-both-egl-9-and-hif-1" data-toc-modified-id="Searching-for-a-TF-that-is-activated-by-both-egl-9-and-hif-1-19"><span class="toc-item-num">19&nbsp;&nbsp;</span>Searching for a TF that is activated by both egl-9 and hif-1</a></div><div class="lev1 toc-item"><a href="#Finding-direct-targets-of-hif-1,-vhl-1,-egl-9-and-rhy-1" data-toc-modified-id="Finding-direct-targets-of-hif-1,-vhl-1,-egl-9-and-rhy-1-20"><span class="toc-item-num">20&nbsp;&nbsp;</span>Finding direct targets of <em>hif-1</em>, <em>vhl-1</em>, <em>egl-9</em> and <em>rhy-1</em></a></div><div class="lev2 toc-item"><a href="#hif-1-direct-targets" data-toc-modified-id="hif-1-direct-targets-20.1"><span class="toc-item-num">20.1&nbsp;&nbsp;</span><em>hif-1</em> direct targets</a></div><div class="lev2 toc-item"><a href="#Quality-control-on-identified-genes:" data-toc-modified-id="Quality-control-on-identified-genes:-20.2"><span class="toc-item-num">20.2&nbsp;&nbsp;</span>Quality control on identified genes:</a></div><div class="lev2 toc-item"><a href="#Identifying-rhy-1-targets" data-toc-modified-id="Identifying-rhy-1-targets-20.3"><span class="toc-item-num">20.3&nbsp;&nbsp;</span>Identifying <em>rhy-1</em> targets</a></div><div class="lev2 toc-item"><a href="#Identifying-egl-9-targets" data-toc-modified-id="Identifying-egl-9-targets-20.4"><span class="toc-item-num">20.4&nbsp;&nbsp;</span>Identifying <em>egl-9</em> targets</a></div><div class="lev2 toc-item"><a href="#Identifying-vhl-1-targets" data-toc-modified-id="Identifying-vhl-1-targets-20.5"><span class="toc-item-num">20.5&nbsp;&nbsp;</span>Identifying <em>vhl-1</em> targets</a></div><div class="lev2 toc-item"><a href="#Identifying-New-Biology---understanding-the-role-of-rhy-1-and-egl-9-in-the-hif-1-dependent-response" data-toc-modified-id="Identifying-New-Biology---understanding-the-role-of-rhy-1-and-egl-9-in-the-hif-1-dependent-response-20.6"><span class="toc-item-num">20.6&nbsp;&nbsp;</span>Identifying New Biology - understanding the role of <em>rhy-1</em> and <em>egl-9</em> in the <em>hif-1</em> dependent response</a></div>

# Genetic Analysis of a Metazoan Pathway using Transcriptomic Phenotypes Supplementary and Extended Material
**David Angeles-Albores, Carmie Puckett Robinson, Brian Williams, Igot Antoshechkin, and Paul W Sternberg**

The purpose of this notebook is to serve as an extended, interactive document for our [paper](http://www.wormbase.org/).

All code in this notebook was written, documented and generated by David Angeles-Albores. In cases when code was inspired, written or shown elsewhere first, links have been provided to the source material where possible.

100% of the data, with the exception of the raw reads, is deposited in our [GitHub project](https://github.com/WormLabCaltech/mprsq). If you would like to verify our analysis, you are welcome to fork the project, although we will not allow pulls into the main branches. I hope you find this useful!

# Folder Structure

The folder structure is a little bit important if you want to replicate the findings presented here. 

Briefly, the folder structure is contained in the following major folders:

 * input - contains raw FASTQ files, kallisto_all (processed reads), genmap file, TF list, cDNA file, transcripts.idx, enrichment dictionaries and hypoxia gold standard gene files
 * sleuth - contains differential analysis (no *fog-2* included) results
 * sleuth_batch_adjusted - contains differential analysis results with *fog-2* included
 * src - all python scripts
 * output - all figures
 * tex - manuscript
 * experimental_docs - all bioanalyzer results are placed here

# Read alignment and differential expression analysis

 The raw FASTQ reads are in input/rawseq/PROJECT_NAME/. These reads are processed by running **kallisto_bash_generator.py**, then from terminal (in the main directory) *chmod +x kallisto commands.sh; sh kallisto_commands.sh*. Reads were processed using a length of **180**bp, with a standard deviation of **60** basepairs, bootstrapped 200 times. The results from this analysis are then placed in input/kallisto_all/PROJECT_NAME/.
 
 Sleuth analysis was performed by running **Sleuth_Prep.py**, then **Sleuth_Command_Writer.py** which generates the folder *sleuth/*. After writing the **diff_exp_analyzer.R** and placing it within the sleuth folder, navigate to sleuth folder using terminal, then type *chmod +x sleuth_commands.sh*, then *sh sleut_commands.sh*. This will perform the differential expression analysis. The results are stored in folders named WT_X, where X is the genotype being compared to wild-type. 
 
 

# Introduction: Genetic Analysis Using Global Expression Measurements

The following sections will provide (excruciating) detail on how we performed the genetic analysis of our mutants using this data. Initially, this analysis was done blindly. For clarity, I have added all the genotype identifiers from the beginning.

To start, we should load all the python libraries that we will need:

In [1]:
# important stuff:
import os
import pandas as pd
import numpy as np

# stats
import sklearn.decomposition
from scipy import stats as sts
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy

# TEA and morgan
import tissue_enrichment_analysis as tea
import morgan as morgan

# network graphics
import networkx as nx

# Graphics
import matplotlib as mpl
import matplotlib.ticker as plticker
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patheffects as path_effects
from matplotlib import rc
rc('text', usetex=True)

# bokeh
import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting

# bayes and mcmc
import pymc3 as pm
import theano

# Display graphics in this notebook
bokeh.io.output_notebook()

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables SVG graphics inline.  There is a bug, so uncomment if it works.
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style("dark")

ft = 35 #title fontsize

Loading BokehJS ...

Next, I need to load the **phenotype_df** (please don't use this file without permission for publication purposes. This tool is unpublished [I am actively working on a paper with this] and will be available soon. I provide it here for clarity, even though it was not part of our paper), the **tissue_df** and the **tf_df**. These files contains a phenotype-gene dictionary, a tissue-gene dictionary (downloaded from TEA) and a transcription factor list (from Chris Grove). 

Additionally, I will initialize several variables whose purpose will mainly be to prettify the tables and substitute in the correct names. 

In [2]:
phenotype_df = pd.read_csv('../input/dictionaries/phenotype_dictionary.csv')
tissue_df = tea.fetch_dictionary()
tf_df = pd.read_csv('../input/tf_list.csv')
hypoxia_gold = pd.read_csv('../input/hypoxia_gold_standard.csv', sep=',')

genotype_mapping = {'a': r'\emph{egl-9;vhl-1}',
                    'f': r'\emph{egl-9;hif-1}',
                    'b': r'\emph{egl-9}',
                    'c': r'\emph{hif-1}',
                    'd': r'\emph{vhl-1}',
                    'e': r'\emph{rhy-1}',
                    'g': r'\emph{fog-2}'
                    }

sort_pairs = {'eb': 1, 'be': 1,
              'ed': 2,'de': 2,
              'ec': 3,'ce': 3,
              'eg': 4,'ge': 4,
              'bd': 5,'db': 5,
              'cb': 6,'bc': 6,
              'bg': 7, 'gb': 7,
              'cd': 8,'dc': 8,
              'dg': 9,'gd': 9,
              'cg': 10,'gc': 10
             }

decode_pairs = {'eb': '\emph{rhy-1}, \emph{egl-9}', 'be': '\emph{rhy-1}, \emph{egl-9}',
              'ed': '\emph{rhy-1}, \emph{vhl-1}','de': '\emph{rhy-1}, \emph{vhl-1}',
              'ec': '\emph{rhy-1}, \emph{hif-1}','ce': '\emph{rhy-1}, \emph{hif-1}',
              'eg': '\emph{rhy-1}, \emph{fog-2}','ge': '\emph{rhy-1}, \emph{fog-2}',
              'bd': '\emph{egl-9}, \emph{vhl-1}','db': '\emph{egl-9}, \emph{vhl-1}',
              'cb': '\emph{egl-9}, \emph{hif-1}','bc': '\emph{egl-9}, \emph{hif-1}',
              'bg': '\emph{egl-9}, \emph{fog-2}', 'gb': '\emph{egl-9}, \emph{fog-2}',
              'cd': '\emph{vhl-1}, \emph{hif-1}','dc': '\emph{vhl-1}, \emph{hif-1}',
              'dg': '\emph{hif-1}, \emph{fog-2}','gd': '\emph{vhl-1}, \emph{fog-2}',
              'cg': '\emph{hif-1}, \emph{fog-2}','gc': '\emph{hif-1}, \emph{fog-2}'
             }

Next, we will load up all of our files. These files will be placed within a class called morgan.hunt (funny!). All of the classes that are referenced in this tutorial are in the file morgan.py, which you are welcome to use for your own analysis. The classes are relatively well-documented and should be usable if you are careful. That said, I am not a computer scientist, so some pathologies or bugs may pop up -- if they do, please perform a pull-request on our github with a fix for the bug. Alternatively, please email me a sufficiently detailed script so I can reconstruct the failure event and correct the bug.

In [3]:
# Specify the genotypes to refer to:
single_mutants = ['b', 'c', 'd', 'e']
# Specify which genotypes are double mutants and of what single mutants:
double_mutants = {'a' : 'bd', 'f':'bc'}

# initialize the morgan.hunt object:
# target_id is the column with isoform specific names
# b is the name of the column with the GLM regression coefficients
# tpm is the name of the column with the TPM numbers
# qval is the name of the column with the FDR corrected q-values
thomas = morgan.hunt('target_id', 'b', 'tpm', 'qval')

# input the genmap file:
thomas.add_genmap('../input/library_genotype_mapping.txt', comment='#')

# add the names of the single mutants
thomas.add_single_mutant(single_mutants)

# add the names of the double mutants
thomas.add_double_mutants(['a', 'f'], ['bd', 'bc'])

# set the q-value threshold for significance to its default value, 0.1
thomas.set_qval()

# Add the tpm files: 
kallisto_loc = '../input/kallisto_all/'
thomas.add_tpm(kallisto_loc, '/kallisto/abundance.tsv', '')

# Make all possible combinations of WT, X
combs = {}
for gene in thomas.genmap.genotype.unique():
    if gene != 'wt':
        combs[gene] = 'WT_'+gene+'/'

# load all the beta values for each genotype:
sleuth_loc = '../sleuth/'
thomas.add_betas(sleuth_loc, 'betas.csv', combs)

Next, I will filter the data, removing any genes that don't show up in all the files and removing the bottom 10% of the genes by expression level. This is an aggressive cutoff, I know. 

In [4]:
thomas.filter_data(0, 0.1)

Number of na genes: 983


## PCA Analysis of TPM Data

PCA analysis of TPM data should hopefully reveal some structure and clustering in this dataset and show us that this isn't pointless immediately. 

First, I want to know how many PCs explain >90% of the data, then build a PCA object with that
many dimensions which I will use for agglomerative clustering. 

In [ ]:
# PCA
#exclude g
pca_matrix = genpy.make_matrix(thomas.tpm, thomas.genmap, 'project_name', 'tpm')
sklearn_pca, n = genpy.pca(pca_matrix)

In [ ]:
# keep only the first principal components that explain 90% of the data
if n < 3:
    n = 3

df_nD = genpy.tidy_pca(pca_matrix, n)

# add a sample and genotype column
df_nD['samples'] = thomas.genmap.project_name.unique()
l = lambda x: thomas.genmap[thomas.genmap.project_name == x].genotype.values[0]
df_nD['genotypes'] = df_nD.samples.apply(l)
df_nD.head()

In [ ]:
def plot_2d_pca(df_pca, group_col, special_key, double_mutant_hash= {},save=''):
    """
    Very basic wrapper around plt.plot to plot pca results after using tidy_pca.
    
    df_pca - df output by tidy_pca
    group_col - column in df_pca to group by
    special_key - if there's a particular group to color in black, use special_key
                  to identify it
    
    returns:
    a matplotlib object
    """
    # plot first 2d components! 
    for key, group in df_nD.groupby(group_col):
        if key in genotype_mapping.keys():
            lab = genotype_mapping[key]
        else:
            lab = key
        if key == special_key:
            plt.plot(group.PCA1, group.PCA2, 'ko', alpha=0.9, label=lab, ms=17)
        else:
            if key not in double_mutant_hash.keys():
                plt.plot(group.PCA1, group.PCA2, 'o', alpha=.9, label=lab, ms=17)
            else:
                plt.plot(group.PCA1, group.PCA2, 'o', alpha=.9, label=lab, ms=17)

    # Tidy up plot
    plt.legend(bbox_to_anchor=(1.2, 1), fontsize=20).set_path_effects([path_effects.Normal()])
    plt.margins(0.05)
    plt.xlabel('PCA 1').set_path_effects([path_effects.Normal()])
    plt.ylabel('PCA 2').set_path_effects([path_effects.Normal()])
    if save:
        plt.savefig(save)
    
    
plot_2d_pca(df_nD, 'genotypes', 'wt', double_mutants)
plt.legend(loc=(1.05, 0.5))

ax = plt.gca()
for i, label in enumerate(ax.get_xticklabels()[::1]):
    ax.get_xticklabels()[::1][i] = ax.get_xticklabels()[::1][i].set_path_effects([path_effects.Normal()])
loc = plticker.MultipleLocator(base=10**4) # this locator puts ticks at regular intervals
ax.xaxis.set_major_locator(loc)
loc = plticker.MultipleLocator(base=5*10**3) # this locator puts ticks at regular intervals
ax.yaxis.set_major_locator(loc)
plt.title('RNA-Seq is a Tool for\nMeasuring Complex Phenotypes', fontsize=ft).set_path_effects([path_effects.Normal()])
plt.savefig('../output/tpm_pca.pdf', bbox_inches='tight')


Clearly, we can see a few things:

*egl-9*, *vhl-1* and *rhy-1* all cluster far away from the WT, primarily along PCA1. 

The double mutant *egl-9; vhl-1* has a mean PCA location very close to *egl-9*. 

The mutant *hif-1* has a centroid relatively close to the WT. 

The double mutant *hif-1* is almost identical to the WT.

However, we are only looking at the first 2 dimensions of the PCA, and we know that 90% of the variance is contained in the first 4. So we can perform agglomerative clustering to make sure these observations are correct. 

# Figure 1. Dendrogram Clustering

Having performed the above analysis, let's go ahead and make the (almost finished) dendrogram figure in the paper. The only post-processing this will undergo is manually changing the letters to gene names and removing some of the labels for clarity.

In [ ]:
# Perform agglomerative clustering
model = hierarchy.linkage(sklearn_pca.transform(pca_matrix), 'ward')

# Draw a canvas:
plt.figure(figsize=(25, 10))

# extract the labels:
labels = df_nD.genotypes.values

# rename the double mutant labels (i.e. instead of 'a' --> 'bd')
for i, label in enumerate(labels):
    if label in double_mutants:
        labels[i] = double_mutants[label]

# Linewidth parameter, temporarily set to 7
plt.rcParams['lines.linewidth'] = 7

# set colors (black; blue)
hierarchy.set_link_color_palette(['k', 'b'])

# draw the dendrogram
hierarchy.dendrogram(
    model,
    truncate_mode='level',  # show only the last p merged clusters
    labels=df_nD.genotypes.values,
    p=21,  # show only the last p merged clusters
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,  # to get a distribution impression in truncated branches
    above_threshold_color='k'
)

# get the current axis
ax = plt.gca()

# add in the orange and green boxes
height = ax.get_ylim()[1]*.36
ax.fill_between([90, 210], 0.0, height, 
                    where=[1,1], color='g', alpha=0.55, zorder=0)
ax.fill_between([0, 90], 0.0, height*1.3, 
                    where=[1,1], color='#e66101', alpha=1, zorder=0)

plt.xticks(fontsize=55)
plt.gca().yaxis.set_major_locator(plt.NullLocator())
plt.title('Clustering by Expression Recapitulates Epistatic Interactions', fontsize=ft*1.5)
plt.savefig('../output/tpm_dendrogram.pdf', bbox_inches='tight')

# return linewidth to a reasonable setting
plt.rcParams['lines.linewidth'] = 2

In fact, it does appear that *hif-1* is closest to the WT out of all the genes in this study. Moreover, please do notice that we can get epistasis relationships out of this data. Great!

## Filtering Does Not Seriously Alter Differentially Expressed Genes

In the section above, I filtered somewhat aggressively. Let's verify that there's not too much change in the number of genes that we detected.

In [ ]:
# go through each dataset
for key, df in thomas.beta.items():
    # before filtering:
    # find the significantly altered genes in each mutant
    sig_genes = len(df[df.qval < thomas.q])
    # calculate the mean fold change (technically a biased estimator of the fold change)
    # as e^(b_mean)
    mc = np.exp(df[df.qval < thomas.q].b.abs().max())
    
    # print the result
    print('{0} has {1} diff. exp. genes before\
 filtering with mean fold change {2:.2g}'.format(key, sig_genes, mc))
    # after filtering:
    df = thomas.beta_filtered[key]
    sig_genes = len(df[df.qval < q])
    print('{0} has {1} diff. exp. genes after filtering'.format(key, sig_genes))
    print('-----------\n')


# Bayesian versus Spearman Regressions

Ok! Having verified that we still have plenty of power to identify these mutants, we need to go ahead and perform some statistics. Next, I will define some useful functions to make plotting easier. After that, I will begin a detailed analysis of the different prediction methods, specifically attempting to compare Spearman rank regression with a Bayesian linear regression of ranked data.

In [ ]:
def pathify(title, xlabel, ylabel, xticks=True, yticks=True, **kwargs):
    """
    A function to pathify the labels, titles and ticks in a plot.
    """
    labelsize = kwargs.pop('labelsize', 20)
    titlesize = kwargs.pop('titlesize', 25)
    
    # make the labels and title into paths
    plt.ylabel(ylabel, fontsize=labelsize).set_path_effects([path_effects.Normal()])
    plt.xlabel(xlabel, fontsize=labelsize).set_path_effects([path_effects.Normal()])
    plt.title(title, fontsize=titlesize).set_path_effects([path_effects.Normal()])

    ax = plt.gca()
    # go through each xtick or ytick and make it a path if user specified to do so.
    if xticks == True:
        for i, label in enumerate(ax.get_xticklabels()):
            ax.get_xticklabels()[i].set_path_effects([path_effects.Normal()])
    if yticks == True:
        for i, label in enumerate(ax.get_yticklabels()):
            ax.get_yticklabels()[i].set_path_effects([path_effects.Normal()])


## Spearman Regression Method

In order to perform the Spearman regression, this is what I will do:

 * Extract the pertinent datasets for the two genotypes I want to convert.
 * Find the genes that are altered between both conditions, regardless of direction
 * After finding this overlap, rank the genes by their $\beta$ coefficient in each dataset
 * Perform Least Squares on the ranked data, which is the Spearman Regression
 * Plot the results

In [ ]:
# lambda index function:
lind = lambda x: (x.qval < 0.1)

# As an example, let me show you what a good spearman correlation looks like:
# name of the column that contains the isoform names:
genes = 'target_id'

# the genotypes to compare
letters = ['e', 'c']

# extract the dataframes from the morgan.hunt object
x = thomas.beta_filtered[letters[0]]
y = thomas.beta_filtered[letters[1]]

# boolean logic to find the stat. sig. diff. genes that appear in both x and y
ovx = x[lind(x)]
ovy = y[lind(y) & y[genes].isin(ovx[genes])].copy()
ovx = x[lind(x) & x[genes].isin(ovy[genes])].copy()

# a function to rank order the data
def find_rank(df):
    """A function to find the rank values of a variable."""
    # make a copy of the dataframe, then sort it inplace
    d = df.copy()
    d.sort_values('b', inplace=True)
    # make a rank vector and append it to the sorted dataframe
    rank = np.linspace(0, len(d)-1, len(d))
    d['r'] = rank
    # sort by isoform name again and return the modified df
    d.sort_values('target_id', inplace=True)
    return d

# apply said function
ovx = find_rank(ovx)
ovy = find_rank(ovy)

# calculate a linear regression on ranked data, which is equivalent to Spearman Ranked Regression
slope, intercept, r_value, p_value, std_err = stats.linregress(ovx.r,ovy.r)

# We make two vectors in order to draw the best fit line on a plot
X = np.linspace(0, len(ovx.r))
Y = slope*X + intercept

# Plot the genes that are statistically significantly altered in both X and Y
plt.plot(ovx.r, ovy.r, 'ro', alpha=1, label='Overlapped Diff. Exp. Genes')

# Plot the best fit line
plt.plot(X, Y, 'k-', alpha=0.6, lw=10, label= 'Best Fit')

# prettify the plot
plt.title('Linear Pathways Share a\nCommon Transcriptomic Phenotype', fontsize=ft).\
            set_path_effects([path_effects.Normal()])
plt.xlabel(genotype_mapping[letters[0]], fontsize=30).set_path_effects([path_effects.Normal()])
plt.ylabel(genotype_mapping[letters[1]], fontsize=30).set_path_effects([path_effects.Normal()])
plt.xticks(fontsize=20)
plt.savefig('../output/spearmanr_b_and_c.pdf')

Spearman R predicts a trend, but the best fit we found doesn't agree with the one I would predict with my eye. We can probably do better if we use a Bayesian regression that minimizes least squares but using a Student-T Prior, NOT a Gaussian. 

## Bayesian Robust Regression
In order to perform the Spearman regression, this is what I will do:

 * Extract the pertinent datasets for the two genotypes I want to convert.
 * Find the genes that are altered between both conditions, regardless of direction
 * After finding this overlap, rank the genes by their $\beta$ coefficient in each dataset
 * Perform Least Squares on the ranked data, but use a Student-T instead of a Gaussian prior.
   * The regression in this case is sampled using a full Monte Carlo Simulation.
 * Identify outliers to this regression, and run a second regression on outliers to see if there are complex regulatory relationships between these genes.
 * Plot the results
 
In other words, given ranked data, I will find the line that is likeliest to explain the data by finding:
\begin{equation}
P(D | \mu, \sigma) \propto \prod_i \mathrm{StudentT}(D_i(x) - \mu(x), \sigma)
\end{equation}

The Student-T distribution has considerably heavier tails than a Gaussian distribution, so it will not consider the evidence provided by outliers as informative as a Gaussian would. 

In order to perform this simulation, we will use the **pymc3** package to specify the model. This model was inspired and successfully deployed thanks to Thomas Wiecki; specifically thanks to [this blog entry](http://twiecki.github.io/blog/2013/08/27/bayesian-glms-2/) by him. Thanks Tom!

## Figure 4. Complex Regulation Generates Detectable Patterns in Transcriptomes

The code that we will run below was used to generate the bottom panel of figure 4 in our paper. The figure as is output here was only subjected to very minor aesthetic modifications post-generation (such as moving the title a little above where it appears here). 

In [ ]:
def robust_regress(data):
    """A robust regression using a StudentT distribution instead of a Gaussian model."""
    with pm.Model() as model_robust:
        # set the model. pymc is nice because it will automatically choose
        # appropriate priors for us once we specify our likelihood is StudentT
        family = pm.glm.families.StudentT()
        # specify we want a generalized linear model with a Student T distribution
        pm.glm.glm('y ~ x', data, family=family)
        # find the MAP as a good starting point
        start = pm.find_MAP()
        # do the simulation and return the results
        step = pm.NUTS(scaling=start)
        trace_robust = pm.sample(2000, step, progressbar=True)
        return trace_robust

In [ ]:
# Take the data from the Spearman example and put it into a dictionary to feed into the robust regression
data = dict(x=ovx.r, y=ovy.r)

x = np.linspace(ovx.r.min(), ovx.r.max())

# perform the simulation
trace_robust = robust_regress(data)

# draw a figure
plt.figure(figsize=(5, 5))

# some statistics.
# normalize everything so that all points are centered around 0
# by taking the y-coordinates and subtracting the value of the model at the point we calculated
distribution = ovy.r - trace_robust.Intercept.mean() - ovx.r*trace_robust.x.mean()
# find the mean and stdev of the distribution (even though mean should be 0 now)
mean = distribution.mean()
std = distribution.std()

# find inliers and outliers (see text description below)
def find_inliers(distribution, mean, trace):
    """A function to identify inliers and outliers in a distribution"""
    distribution_inliers = distribution[np.abs(distribution - mean)/(trace_robust.x.std() 
                                                                 + trace_robust.Intercept.std()+ std) < 1]
    distribution_outliers = distribution[np.abs(distribution - mean)/(trace_robust.x.std() 
                                                                 + trace_robust.Intercept.std()+ std) > 1]

    # get the gene names of the outliers
    outliers = ovy[ovy.r.isin(distribution_outliers + 
                          trace_robust.Intercept.mean() + ovx.r*trace_robust.x.mean())].target_id
    return distribution_inliers, distribution_outliers, outliers

# call the function
distribution_inliers, distribution_outliers, outliers = find_inliers(distribution, mean, trace_robust)

# run a secondary regression on the outliers
data2 = dict(x=ovx[ovx.target_id.isin(outliers)].r, y=ovy[ovy.target_id.isin(outliers)].r)
trace_robust2 = robust_regress(data2)

# get the y-coordinate of the inliers and outliers
yri = distribution_inliers + trace_robust.Intercept.mean() + ovx.r*trace_robust.x.mean()
yro = distribution_outliers + trace_robust.Intercept.mean() + ovx.r*trace_robust.x.mean()

# plot the regression lines
pm.glm.plot_posterior_predictive(trace_robust, eval=x, 
                                 label='posterior predictive regression lines',
                                 color='#357EC7')
pm.glm.plot_posterior_predictive(trace_robust2, eval=x, 
                                 label='posterior predictive regression lines',
                                 color='#FFA500')

# plot the data. Inliers are plotted as large green dots, outliers as small red dots
plt.plot(ovx.r, yri, 'go', ms = 7.5)
plt.plot(ovx[yro > 0].r, yro[yro > 0], 'ro', ms = 5)

# prettify plot
plt.xlim(0, len(ovx))
plt.ylim(0, len(ovy))
plt.yticks([0, 200, 400], fontsize=22)
plt.xticks([0, 200, 400], fontsize=22)
pathify('Transcriptome Reflects Multiple Interaction Modes', r'\emph{rhy-1} genes ranked by $\beta$', 
        r'\emph{hif-1} genes ranked by $\beta$', labelsize=24)
plt.savefig('../output/multiple_modes_{0}.pdf'.format(letters[0]+letters[1]))

There, much better! Penalizing outliers allows us to estimate the primary regression slope really well (blue lines). Here, each blue line is the result of a simulation, and we can see that they are all clustered together. Having estimated the first regression, we can proceed to identify outliers. 

Now, there are multiple ways to identify outliers, and this is the point where I am going to cheat just a little bit. If we pretend that the model is gaussian (it isn't), then we can identify outliers (fairly aggressively) via the following method. Thus, if

\begin{equation}
z(i) = \frac{D_i(x) - \mu(x)}{\sigma_\mu + \sigma_{\mathrm{Intercept}} + \sigma_\mathrm{Data}} > 1
\end{equation}

then $z(i)$ is an outlier. 

After identifying these outliers, I can then pool them and run a second regression on them (orange lines). These lines are also quite clustered, and the slope is practically 1! Wow!


## Figure 2. Positive Regulatory Relationships Can Be Identified By Transcriptomic Correlation

In the above example, we saw that *rhy-1* and *hif-1* share a complex regulatory relationship as exhibited by the cross-pattern. However, an important point is whether or not this cross pattern appears in all datasets. While we cannot generalize yet (this kind of analysis is very new) we should definitely make sure that not all of our genes are exhibiting this cross pattern. Below, I show some a pair of genes that don't exhibit a cross pattern.

In [ ]:
genes = 'target_id'

letters = ['b', 'd']

# datasets
x = thomas.beta_filtered[letters[0]]
y = thomas.beta_filtered[letters[1]]

# overlap
ovx = x[lind(x)]
ovy = y[lind(y) & y[genes].isin(ovx[genes])].copy()
ovx = x[lind(x) & x[genes].isin(ovy[genes])].copy()

# find rank
ovx = find_rank(ovx)
ovy = find_rank(ovy)

# Set up MCMC parameters
data = dict(x=ovx.r, y=ovy.r)

x = np.linspace(ovx.r.min(), ovx.r.max())
    
# simulation
trace_robust = robust_regress(data)

# canvas
plt.figure(figsize=(5, 5))

# statistics
distribution =ovy.r - trace_robust.Intercept.mean() - ovx.r*trace_robust.x.mean()
mean = distribution.mean()
std = distribution.std()

# find the inliers and outliers
distribution_inliers, distribution_outliers, outliers = find_inliers(distribution, mean, trace_robust)

# y-coordinate of outliers
yri = distribution_inliers + trace_robust.Intercept.mean() + ovx.r*trace_robust.x.mean()
yro = distribution_outliers + trace_robust.Intercept.mean() + ovx.r*trace_robust.x.mean()

# plot primary regression
pm.glm.plot_posterior_predictive(trace_robust, eval=x, 
                                 label='posterior predictive regression lines',
                                 color='#357EC7')

# plot inliers and outliers
plt.plot(ovx.r, yri, 'go', ms = 7.5)
plt.plot(ovx[yro > 0].r, yro[yro > 0], 'ro', ms = 5)

# prettify
plt.xlim(0, len(ovx))
plt.ylim(0, len(ovy))
plt.yticks([0, 200, 400], fontsize=20)
plt.xticks([0, 200, 400], fontsize=20)
pathify('Positive Genetic Interactions', r'\emph{egl-9} genes ranked by $\beta$', 
        r'\emph{vhl-1} genes ranked by $\beta$')
plt.savefig('../output/positive interaction{0}.pdf'.format(letters[0]+letters[1]))

# Pairwise Analysis of All Genes Using Spearman Correlation

Given what we found above, we know that the spearman analysis is good to use to get a rough fast idea of how our data looks. Let's go ahead and run the analysis for all pairwise comparisons that can be made using this dataset. 

Spearman correlations and hypergeometric tests can be performed using the class morgan.brenner. 

Once I calculate the quantities of interest, I will plot them on a heatmap. However, reader beware. Do NOT **EVER** use jet. The red and blue heatmaps? Awful. They have very serious pathologies that have been well described, and they generate patterns via optical illusions. Instead, we will use the *viridis* colormap, which was used by scientists in LIGO.

In [ ]:
# Define a 'genes' variable that will be used for labelling every plot from here on out:
genes = [genotype_mapping[x] for x in thomas.single_mutants]

# Define a plotting function to plot only a triangular heat map
def tri_plot(matrix, xlabels, ylabels=[]):
    """Given a matrix, a set of xlabels and ylabels, draw a triangle plot."""
    # Minimum and maximum for colormap
    vmin= matrix.min().min()
    vmax= np.max(matrix).max()

    # if user didn't specify xlabels, assume ylabels are the same as xlabels
    if len(ylabels) == 0:
        ylabels = xlabels

    # make the lower triangle of the matrix, since we are only dealing with 
    # symmetric matrices. Also, remove the diagonal
    mask = np.zeros_like(matrix)
    mask[np.tril_indices_from(mask)] = True

    # draw and adjust xtick size
    with sns.axes_style("white"):
        ax = sns.heatmap(matrix, xticklabels=xlabels, yticklabels=ylabels, cmap='viridis',
                         mask=mask, square=True, vmin=vmin, vmax=vmax)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)

# Perform Correlation Analysis
sydney = morgan.brenner('spearman', thomas)
# Plot
tri_plot(sydney.rho.as_matrix(columns=thomas.single_mutants), genes)

So maybe *egl-9* and *rhy-1* interact? Let's wait for the full Bayesian treatment.

# Hypergeometric Analysis:

In [ ]:
mat = sydney.hyper_minus.as_matrix(columns=thomas.single_mutants)
genes = [genotype_mapping[x] for x in thomas.single_mutants]
tri_plot(mat, genes)
pathify('Negative Genetic Interaction\nProbability', '', '')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.savefig('../output/probability_of_inhibition_single_mutants.pdf', bbox_inches='tight')

# Analysis Using Robust Regression

What I do here is try to identify trends. Namely, for two genotypes X and Y, I fit a line using a Bayesian robust regression (see methods) thru the rank-ordered regression coefficients that are statistically significantly different from 0 and that are present in both X and Y. Next, I use this Bayesian framework to identify any and all outliers to the regression. 

In order to test for alternative modes of interaction in this dataset, I take the outliers and I run the same regression again *on the outliers*. If this second interaction has an opposite sign to the first, then we predict that there are two modes of interaction, subject to the following caveats:

 * Both the primary and secondary regressions yield strong correlations (>0.7)
 * The first regression results in many outliers

Finally, because of the approach I used, I can also predict which genes are under what mode of regulation. Super cool!

In [ ]:
barbara = morgan.mcclintock('bayesian', thomas, True)

The simulation has finished, and now I should make the heat map showing the primary correlations.

## Figure 3. Pairwise regression values between all single mutants


In [ ]:
mat = barbara.robust_slope.as_matrix(columns=thomas.single_mutants)
tri_plot(mat, genes)
plt.xticks(rotation=45)
plt.yticks(rotation=45)
pathify('Robust Bayesian Correlations Predict Interaction Between Genes', '', '')
plt.savefig('../output/bayes_primary_single_mutants.pdf', bbox_inches='tight')

We can also plot the secondary correlations and see what comes out of them

In [ ]:
mat = barbara.secondary_slope.as_matrix(columns=thomas.single_mutants)
tri_plot(mat, genes)
plt.title('Robust Bayesian Correlations Predict Interaction Between Genes', fontsize=20)


# Extracting functional relationships between genes

Another way to plot the results from the regression above is to make scatterplots. For that, it's best to work with tidy dataframes. 
What I will do next is the following:

Take the square correlation matrix I calculated above. Place it in a [tidy dataframe](http://vita.had.co.nz/papers/tidy-data.pdf). Concatenate this dataframe to include both primary and secondary relationships. Add a column that contains an indicator variable specifying whether the primary and the secondary regressions have different sign. If they don't have different signs, then add the correlations together. The reasoning for this is that:

\begin{equation}
w_1\rho + w_2\rho = (w_1+w_2)\rho
\end{equation}

In other words, we expect the correlation coefficient to be the same between the primary and secondary correlation and performing this is heuristically the same as adding the weights together. I know there's a better way to do this, and I am actively working on implementing this is in a proper manner. In the future, the weights will be added, and the secondary regression wil be discarded. 

Finally, I order the pairs of genes to reflect decreasing functional distance and plot. 

In [ ]:
def tidy_df(df, corr='corr'):
    """
    A function that returns a tidied up dataframe.
    
    Dataframe provided must be the result of morgan.robust_regression()
    or morgan.robust_regression_secondary()
    
    df - dataframe to tidy up
    corr - please specify whether this dataframe is from morgan.robust_regression (write 'corr')
    or from morgan.robust_regression_secondary() (write 'outlier')
    
    outputs:
    df - a tidied dataframe with columns 'corr_wit', 'variable', 'fraction' and 'pair'
    """
    df = df.copy()
    df['corr_with'] = thomas.single_mutants
    df = pd.melt(df, id_vars='corr_with')
    df = df[df.corr_with != df.variable]

    def add_fraction_outliers(x, fraction='corr'):
        if (x.corr_with, x.variable) in barbara.correlated_genes.keys():
            dd = barbara.correlated_genes[(x.corr_with, x.variable)]
            outliers = len(dd['outliers'])
            corr = len(dd['corr'])
            total = outliers + corr
            if fraction == 'corr':
                return corr/total
            else:
                return outliers/total
        else:
            return np.nan

    df['fraction_outliers'] = df.apply(add_fraction_outliers, args=(corr,), axis=1)
    df['pair'] =  df.variable + df.corr_with
    return df

def different(x):
    p = x.pair
    primary = d[(d.pair == p) & (d.regression == 'primary')].value.values[0]
    secondary = d[(d.pair == p) & (d.regression == 'secondary')].value.values[0]
    
    if primary == 0 or secondary == 0:
        return 0
    elif (primary*secondary > 0):
        return -1
    else:
        return 1

def special_add(x):
    if x.regression == 'secondary':
        if x.different == -1:
            return np.nan
        else:
            return x.value
    if d[(d.regression=='secondary') & (d.pair == x.pair)].different.values == -1:
        to_add = d[(d.regression=='secondary') & (d.pair == x.pair)].value.values[0]
        return x.value + to_add
    else:
        return x.value

d_pos = tidy_df(barbara.robust_slope)
d_pos['regression'] = 'primary'
d_minus = tidy_df(barbara.secondary_slope, corr='outliers')
d_minus['regression'] = 'secondary'

## Figure 4. Weighted Correlations Reflect Functional Distance

Now we can plot the weighted correlations, making sure that they are ordered by functional distance.

In [ ]:
frames = [d_pos, d_minus]
d = pd.concat(frames)
d['different'] = d.apply(different, axis=1)
d.dropna(subset=['fraction_outliers'], inplace=True)
d['corrected'] = d.apply(special_add, axis=1)
d.dropna(subset=['corrected'], inplace=True)
d['sort_pairs'] = d.pair.map(sort_pairs)
d.sort('sort_pairs', inplace=True)
d['genes'] = d.pair.map(decode_pairs)
sns.stripplot(x='genes', y='corrected', data=d[d.regression=='primary'], size=20, color='g')
plt.xticks(rotation=45, fontsize=20)
plt.yticks([-0.1, 0, 0.5], fontsize=20)
plt.axhline(0, lw=2, ls='--', color='gray')
pathify('Weighted Correlations Reflect Functional Distance',
        'Pairs of Genes, Ordered by Decreasing Functional Distance',
       'Primary Correlation, Normalized to Overlap')
plt.savefig('../output/weighted_corr_decreases_w_distance.pdf')

In [ ]:
sns.stripplot(x='genes', y='corrected', 
              data=d[(d.regression=='secondary') & (d.different == 1)], size=10, color='k')
plt.axhline(0, ls='--', color='0.5')
plt.xticks(rotation=45, fontsize=20)
plt.yticks([-0.1, 0, 0.1], fontsize=20)
plt.axhline(0, lw=2, ls='--', color='gray')
plt.ylabel('Secondary Correlation, Normalized to Overlap')

It appears that 'C' and 'B' and 'C' and 'E' interact in both a positive and negative manner and both correlations are well supported by evidence. Intriguing!

# Double Mutant Analysis

With double mutants, the analysis gets slightly more complicated. Now we're getting into full pathways!

A first approach is to inspect the double mutants by Spearman correlation analysis to the single mutants that make them up. A quick visualization will show us any epistasis and the extent of it:

In [ ]:
alfred = morgan.sturtevant('epistasis analysis')
alfred.epistasis_analysis(thomas)
alfred.epistasis_secondary(thomas, True)

f e
Applied log-transform to lam and added transformed lam_log_ to model.
 [-----------------100%-----------------] 2000 of 2000 complete in 1.7 secf d
Applied log-transform to lam and added transformed lam_log_ to model.
 [-----------------100%-----------------] 2000 of 2000 complete in 1.6 secf c
Applied log-transform to lam and added transformed lam_log_ to model.
 [-----------------100%-----------------] 2000 of 2000 complete in 1.5 secf b
Applied log-transform to lam and added transformed lam_log_ to model.
 [-----------------100%-----------------] 2000 of 2000 complete in 1.5 seca e
Applied log-transform to lam and added transformed lam_log_ to model.
 [-----------------100%-----------------] 2000 of 2000 complete in 2.3 seca d
Applied log-transform to lam and added transformed lam_log_ to model.
 [-----------------100%-----------------] 2000 of 2000 complete in 1.9 seca c
Applied log-transform to lam and added transformed lam_log_ to model.
 [-----------------100%---------------

In [ ]:
alfred.epistasis['double_corr'].shape

In [ ]:
alfred.epistasis['unweighted'] = alfred.epistasis.correlation/alfred.epistasis.weights


alfred.epistasis['double mutant'] = alfred.epistasis.double_mutant.map(genotype_mapping)
alfred.epistasis['corr with'] = alfred.epistasis.corr_with.map(genotype_mapping)

sns.stripplot(x='double mutant', y='unweighted', hue='corr with', data=alfred.epistasis, size=15, jitter=True)
plt.xticks(fontsize=20)
plt.axhline(0, ls='--', color='0.5')
plt.yticks([-0.2, 0.5, 1], fontsize=20)

In [ ]:
sns.stripplot(x='double mutant', y='correlation', hue='corr with', data=alfred.epistasis, size=15, jitter=True)
plt.xticks(fontsize=20)
plt.axhline(0, ls='--', color='0.5')
plt.yticks([-0.1, 0.2, 0.4], fontsize=20)

In [ ]:
alfred.epistasis_secondary['unweighted'] = alfred.epistasis_secondary.correlation/alfred.epistasis_secondary.weights
alfred.epistasis_secondary['fractional corr'] = alfred.epistasis_secondary.correlation/alfred.epistasis.correlation.abs()

alfred.epistasis_secondary['double mutant'] = alfred.epistasis_secondary.double_mutant.map(genotype_mapping)
alfred.epistasis_secondary['corr with'] = alfred.epistasis_secondary.corr_with.map(genotype_mapping)


sns.stripplot(x='double mutant', y='unweighted', hue='corr with', data=alfred.epistasis_secondary,
              size=15, jitter=True)
plt.xticks(fontsize=20)
# plt.ylim(-1.1, 1.1)
# plt.axhline(0, ls='--', color='0.5')
# plt.yticks([-1, 0, 1], fontsize=20)

In [ ]:
sns.stripplot(x='double mutant', y='correlation', hue='corr with', data=alfred.epistasis_secondary,
              size=15, jitter=True)
plt.xticks(fontsize=20)
plt.ylim(-1.1, 1.1)
plt.axhline(0, ls='--', color='0.5')
plt.yticks([-1, 0, 1], fontsize=20)

In [ ]:
import scipy
import statsmodels.api as sm

def quadrature(x):
    def square(x):
        return x**2
    return np.sqrt(np.sum(x.apply(square)))

double = 'f'
for key in barbara.correlated_genes.keys():
    pair = key[0] + key[1]
    if pair not in d.pair.unique():
        pair = key[1] + key[0]

    pred_genes = barbara.correlated_genes[key]['corr']
    if (pair != thomas.double_muts[double]):
        continue
    
    df_pred = thomas.beta[key[0]].copy()
    df_pred = df_pred[df_pred.target_id.isin(pred_genes)]
    df_test = thomas.beta[double]
    df_test = df_test[df_test.target_id.isin(pred_genes) & (df_test.qval< 0.1)].copy()
    df_pred = df_pred[df_pred.target_id.isin(df_test.target_id)]
    df_test = find_rank(df_test)
    df_pred = find_rank(df_pred)
    
    df_ultimate = thomas.beta[key[1]].copy()
    df_ultimate['b_2'] = thomas.beta[key[0]].b
    df_ultimate['b'] = (df_ultimate.b + df_ultimate.b_2)/2
    df_ultimate = df_ultimate[df_ultimate.target_id.isin(pred_genes)]
    df_ultimate = df_ultimate[df_ultimate.target_id.isin(df_test.target_id)]
    df_ultimate = find_rank(df_ultimate)
    print(len(df_ultimate), len(df_test))
    
    plt.errorbar(df_ultimate.b, df_test.b - df_ultimate.b,
                 xerr=df_ultimate.se_b, yerr=df_test.se_b, fmt='go', alpha=0.2, ms=4)
    plt.errorbar(df_pred.b, df_test.b - df_pred.b,
                 xerr=df_ultimate.se_b, yerr=df_test.se_b, fmt='bo', alpha=0.2, ms=4)
    
    lr = scipy.stats.linregress(df_ultimate.b, df_test.b)
    lr2 = scipy.stats.linregress(df_pred.b, df_test.b)
    
    wls = sm.WLS(df_test.b- df_ultimate.b, df_ultimate.b, weights=1. / df_ultimate.se_b**2)
    res_wls = wls.fit()
    
    wls2 = sm.WLS(df_test.b- df_pred.b, df_pred.b, weights=1. / df_pred.se_b**2)
    res_wls2 = wls2.fit()
    
    slope = res_wls.params
    slope2 = res_wls2.params

    print(key[1], res_wls.summary())
    print(key[0], res_wls2.summary())

    y = x*slope.b
    y2 = x*slope2.b
    plt.plot(x, y, 'g-', lw=2)
    plt.plot(x, y2, 'b-', lw=2)
    prediction = df_ultimate.b + lr.intercept
    mnh = scipy.stats.linregress(df_ultimate.b, df_test.b - prediction)

    plt.xlim(-8,8)
    plt.ylim(-8,8)
    # plt.xscale('symlog')
    # plt.yscale('symlog')
    print(key[1], lr)
    print(key[0], lr2)
    print(df_pred.b.abs().mean())
    print(
"""
{0}: mean b = {3:.2g} +/- {6:.2g}
{1}: mean b = {4:.2g} +/- {7:.2g}
{2}: mean b = {5:.2g} +/- {8:.2g}
""".format(key[0], key[1], pair,
          df_pred.b.abs().mean(), df_ultimate.b.abs().mean(), df_test.b.abs().mean(),
          (df_pred.b.abs().std() + quadrature(df_pred.se_b))/np.sqrt(len(df_pred)),
          (df_ultimate.b.abs().std() + quadrature(df_pred.se_b))/np.sqrt(len(df_pred)),
          (df_test.b.abs().std() + quadrature(df_pred.se_b))/np.sqrt(len(df_pred)),
          )
)

In [ ]:
for key, value in thomas.double_muts.items():
    x = thomas.beta_filtered[key]
    y = thomas.beta_filtered[value[0]]
    z = thomas.beta_filtered[value[1]]
    
    x = x[x.qval < thomas.q]
    y = y[y.qval < thomas.q]
    z = z[z.qval < thomas.q]
    
    yANDz = len(y[y.target_id.isin(z)])
    yORz = len(y) + len(z)
    expected = yORz - yANDz
    
    pred1 = x[(x.target_id.isin(z.target_id))]
    pred2 = x[x.target_id.isin(y.target_id)]
    
    pred = len(list(set(pred1.target_id.tolist() + pred2.target_id.tolist())))
    
    print('Expected: ', expected)
    print('Observed: ', len(x))
    print('Predicted: ', pred)
    print('Observed/Expected: {0:.2g}'.format(len(x)/expected))
    print('Predicted/Expected: {0:.2g}'.format(pred/expected))
    print('Surprise factor: {0:.2g}'.format(pred/len(x)))

Double mutants:
$$
a = b^-d^-
$$

$$
f = b^-c^-
$$

In [ ]:
letters = ['a', 'c']

x = thomas.beta_filtered[letters[0]]
y = thomas.beta_filtered[letters[1]]

ovx = x[lind(x)]
ovy = y[lind(y) & y.target_id.isin(ovx.target_id)].copy()
ovx = x[lind(x) & x.target_id.isin(ovy.target_id)].copy()

ovx = find_rank(ovx)
ovy = find_rank(ovy)

data = dict(x=ovx.r, y=ovy.r)

x = np.linspace(ovx.r.min(), ovx.r.max())
    
trace_robust = robust_regress(data)
plt.figure(figsize=(5, 5))

distribution =ovy.r - trace_robust.Intercept.mean() - ovx.r*trace_robust.x.mean()
mean = distribution.mean()
std = distribution.std()

# find the inliers and outliers
distribution_inliers, distribution_outliers, outliers = find_inliers(distribution, mean, trace_robust)

data2 = dict(x=ovx[ovx.target_id.isin(outliers)].r, y=ovy[ovy.target_id.isin(outliers)].r)
trace_robust2 = robust_regress(data2)


yri = distribution_inliers + trace_robust.Intercept.mean() + ovx.r*trace_robust.x.mean()
yro = distribution_outliers + trace_robust.Intercept.mean() + ovx.r*trace_robust.x.mean()

plt.plot(ovx.r, yri, 'go', ms = 6)
plt.plot(ovx.r, yro, 'ro', ms = 5)
pm.glm.plot_posterior_predictive(trace_robust, eval=x, 
                                 label='posterior predictive regression lines',
                                 color='#357EC7')
pm.glm.plot_posterior_predictive(trace_robust2, eval=x, 
                                 label='posterior predictive regression lines',
                                 color='#FFA500')
plt.xlim(0, len(ovx))
# plt.legend()

if np.abs(trace_robust.x.mean()) > 0.6 and np.abs(trace_robust2.x.mean()) > 0.6:
    t1 = trace_robust.x.mean()/np.abs(trace_robust.x.mean())
    t2 = trace_robust2.x.mean()/np.abs(trace_robust2.x.mean())
    if t1 == -t2:
        print('Complex Regulation at Work')

# Defining the Hypoxia Response

In [ ]:
# find all the genes in the hypoxia response:
ovb = thomas.beta_filtered['b'].copy()
ovd = thomas.beta_filtered['d']
ove = thomas.beta_filtered['e']

ovb['d_qval'] = ovd.qval
ovb['e_qval'] = ove.qval

ovb['d_b'] = ovd.b
ovb['e_b'] = ove.b

# qvalue cutoff
ind1 = (ovb.qval < 0.1)
ind2 = (ovb.d_qval < 0.1)
ind3 = (ovb.e_qval < 0.1)

# make sure they all go in the same direction:
ind4 = (ovb.b*ovb.d_b > 0) & (ovb.b*ovb.e_b > 0) & (ovb.d_b*ovb.e_b >0)

hypoxia_genes = ovb[ind1 & ind2 & ind3 & ind4].copy()

In [ ]:
len(hypoxia_genes)

In [ ]:
tissue_df = tea.fetch_dictionary()
df_res, unused = tea.enrichment_analysis(hypoxia_genes.ens_gene, tissue_df, save=False)

In [ ]:
hypoxia_genes['avg_qval'] = (hypoxia_genes.qval + hypoxia_genes.e_qval + hypoxia_genes.d_qval)/3
hypoxia_genes['avg_b'] = (hypoxia_genes.b + hypoxia_genes.d_b + hypoxia_genes.e_b)/3
hypoxia_genes.sort_values(['avg_qval'], inplace=True)
hypoxia_genes.to_csv('../output/hypoxia_high_confidence.csv', index=False)

# The genes upregulated by *egl-9* should be down in *hif-1*.

However, egl-9 and hif-1 seem to have a positive primary relationship under normoxic conditions. Are the secondary genes associated with the hypoxia response?

In [ ]:
# First, identify the genes that are outliers in the bc, ec response and intersect them.
if ('c', 'b') in barbara.correlated_genes.keys():
    key = ('c', 'b')
else:
    key = ('b', 'c')

if ('c', 'e') in barbara.correlated_genes.keys():
    key2 = ('c', 'e')
else:
    key2 = ('e', 'c')

print(key)
bc_in = barbara.correlated_genes[key]['corr']
ec_in = barbara.correlated_genes[key2]['corr']
bec_in = np.intersect1d(bc_in, ec_in)
print("""
There are {0} genes that are inliers in the BC intersection,
There are {1} genes that are inliers in the EC intersection,
and there are {2} genes in the BEC inliers intersection.
""".format(len(bc_in), len(ec_in), len(bec_in)))


bc = barbara.correlated_genes[key]['outliers'].target_id
ec = barbara.correlated_genes[key2]['outliers'].target_id
bec = np.intersect1d(bc, ec)
print("""
There are {0} genes that are outliers in the BC intersection,
There are {1} genes that are outliers in the EC intersection,
and there are {2} genes in the BEC outlier intersection.
""".format(len(bc), len(ec), len(bec)))

print(
"""
{0:.2g} of the genes in the BEC intersection are outliers
""".format(len(bec)/(len(bec) + len(bec_in)))
)

In [ ]:
intersection = hypoxia_genes[hypoxia_genes.target_id.isin(bec)].ext_gene.unique()
print(
"""
There are {} genes from the BEC intersection in the hypoxia set!
""".format(len(intersection))
)

In [ ]:
L = thomas.beta_filtered['a'].shape[0] # total genes measured for any experiment

p = stats.hypergeom.sf(len(intersection), L, len(bec), len(hypoxia_genes))

if p < 10**-3:
    print(
"""
This result is statistically significant, with p-value = {0:.2g}
""".format(p)
    )

In fact, it does seem that hypoxia related genes are under control of *hif-1* under normoxic conditions, and that knocking *hif-1* out in a normoxic animal causes changes in this gene set. However, this is not the PRIMARY role of *hif-1* under physiologic conditions

In [ ]:
intersection

In [ ]:
temp = thomas.beta_filtered['c']
temp[temp.target_id.isin(intersection)].qval.values

Unfortunately, none of them are immediately associated with the hypoxia pathway, but a number of these are stress response genes! Thankfully, the q-values are also reasonably low for most of these genes. We're all good. 

# Quality Control

Load up the hypoxia gold standard Carmie gave me. These genes are expected to go up in everything except in Hif-1. 

By this point, I have solved the network, I guessed which gene was Hif-1 and I have been told the identities of the genes. 

C = Hif-1  
B = egl-9  
D = vhl-1  
E = rhy-1  

nhr-57 is an important gene for hypoxia. WBID is WBGene00003647

In [ ]:
# Figure out how many hits we get and if the pvalue is significant!
for sm in thomas.single_mutants:
    df = thomas.beta_filtered[sm]
    ind = (genpy.find(df, hypoxia_gold.WBIDS, col='ens_gene'))
    found = df[ind & (df.qval < 0.1)]
    sig = df[df.qval < 0.1]
    pval = stats.hypergeom.sf(len(found), len(df), len(hypoxia_gold), len(sig))
    print('genotype: ', sm)
    print('found: ', len(found), '      Mean b: {0:.2g}'.format(found.b.mean()))
    print('pval: {0:.2g}'.format(pval))
    
    
    print('Maximum change was:')
    print(found[found.b == found.b.max()].ext_gene.values[0])
    print(np.exp(found[found.b == found.b.max()].b.values)[0], ' fold change')
    if 'WBGene00003647' in found.ens_gene.values:
        nhr57 = np.exp(found[found.ens_gene == 'WBGene00003647'].b.values[0])
        print('nhr-57 is in', sm, 'and its fold change was {0:.2g}'.format(nhr57))
    
    print('----------')

# Correlation Graph

In [ ]:
G=nx.Graph()

mutant_dict = {'b': 'egl-9',
               'c': 'hif-1',
               'e': 'rhy-1',
               'd': 'vhl-1',
               'g': 'fog-2'
              }

for i, key in enumerate(thomas.single_mutants):
    for j, key2 in enumerate(thomas.single_mutants):
        df = d
        r = d[(d.variable == key) & (d.corr_with == key2) &
              (d.regression == 'primary')].corrected.values
        if r:
            G.add_edge(mutant_dict[key], mutant_dict[key2], weight=r[0])

elarge=[(u,v) for (u,v,d) in G.edges(data=True)]
width=[15*d['weight'] for (u,v,d) in G.edges(data=True)]
weights=[d['weight'] for (u,v,d) in G.edges(data=True)]

fig, ax = plt.subplots()

pos=nx.spring_layout(G) # positions for all nodes

# nodes
nx.draw_networkx_nodes(G, pos, node_size=1500, node_color='g', alpha=.5)

# edges
edges = nx.draw_networkx_edges(G, pos, edgelist=elarge,
                       width=width, edge_color=weights,
                       edge_cmap=plt.cm.viridis)

# labels
nx.draw_networkx_labels(G, pos, font_size=15, font_family='sans-serif')

fig.colorbar(edges)

# plt.axis('off')
plt.xticks([])
plt.yticks([])
plt.savefig("../output/weighted_graph.pdf") # save as png
plt.show() # display

# An *in silico* qPCR experiment:

In [ ]:
sorter = {'a': 6,
          'f': 7,
          'b': 2,
          'c': 4,
          'd':3,
          'e': 1,
          'g': 5
         }

x = ['WBGene00001851',
     'WBGene00012324',
     'WBGene00001178',
     'WBGene00006922',
     'WBGene00003647',
     'WBGene00002248'
    ]

# run the experiment!
def qPCR_prep(morgan, genelist):
    g = []
    data = np.array([])
    i = 0
    for genotype, df in thomas.beta.items():
        for j, xi in enumerate(genelist):
            geno = genotype_mapping[genotype]
            y = df[(df.ens_gene == xi)][['ens_gene', 'ext_gene','b', 'se_b', 'qval']].values
            if len(y) == 0:
                continue
            # hif has two isoforms, so take F38A6.3c
            if y.shape[0] > 1:
                y = df[(df.target_id == 'F38A6.3c')][['ens_gene', 'ext_gene','b', 'se_b', 'qval']].values
            if len(data) == 0:
                data = y
            else:
                data = np.vstack((data, y))
            g += [genotype]
        i += 1

    d = pd.DataFrame(data, columns=['ens_gene', 'ext_gene', 'b', 'se_b', 'qval'])
    d['code'] = g
    d['genotype'] = d.code.map(genotype_mapping)
    d['order'] = d.code.map(sorter)
    d.sort_values('order', inplace=True)
    d.reset_index(inplace=True)  
    return d

d = qPCR_prep(thomas, x)

In [ ]:
# a qPCR barplot
temp = d
index = np.linspace(0, temp.genotype.unique().shape[0]-1, temp.genotype.unique().shape[0])
alpha = 0.7
error_config = {'ecolor': '0.2'}

plotting = {'rhy-1': 0,
            'egl-9': 1,
            'hif-1': 2,
            }

color = {'rhy-1': "#ca0020",
         'egl-9': '#f4a582',
         'hif-1': '#f7f7f7',
         'nhr-57':'#92c5de',
         'lam-3': '#0571b0'
        }

grouped = temp.groupby('ext_gene')

bar_width = 1/(len(grouped)+1)

for name, group in grouped:
    if name not in plotting.keys():
        where = max(plotting.keys(), key=lambda k: plotting[k])
        val = plotting[where]
        plotting[name] = val + 1
    
    add = plotting[name]*bar_width
    if name in color.keys():
        barlist = plt.bar(index + add, group.b.values, bar_width, alpha=alpha,
                          yerr=group.se_b, error_kw=error_config, label=name,
                          color=color[name])
    else:
        barlist = plt.bar(index + add, group.b.values, bar_width, alpha=alpha,
                          yerr=group.se_b, error_kw=error_config, label=name)
    sig = group.qval < 0.1
    k = group[sig].order -1 
    plt.plot(k + add + bar_width/2, group[sig].b.values + group[sig].se_b.values + 0.25, r'*', color='k')
#     plt.plot(k + add + bar_width/2, np.repeat(temp.b.max() + 0.25, len(k)), r'*', color='k')

grouped2 = temp.groupby('genotype')
k = 0
col = '#CFCFCF'
for name, group in grouped2:
    if k % 2 == 0:
        xmin = k - bar_width*0.5
        xmax = k + bar_width*(len(grouped) + 0.5)
        ymin, ymax = plt.gca().get_ylim()
        plt.fill_between([xmin, xmax], ymax, color=col)
        plt.fill_between([xmin, xmax], ymin, color=col)
    k += 1

plt.xlim(0, plt.gca().get_xlim()[1] - bar_width)
plt.tick_params(axis='y', which='major', labelsize=18)
plt.xticks(index + bar_width, temp.genotype.unique(), rotation=45, fontsize=20)
pathify(r'\emph{In Silico} qPCR', '', r'Regression Coefficient, $\beta$', )
plt.legend(loc=(1.02, 0.5), fontsize=15)
plt.savefig('../output/pathwaygenes_qPCR.pdf', bbox_inches='tight')

# Define the Absolute Gold Set 
by Finding the Intersection Between the Double Mutant that is Constitutively Hypoxic and the single mutants that are also hypoxic

In [ ]:
# gene should be in double constitutive:
temp = thomas.beta_filtered['a']
temp = temp[temp.qval < 0.1]
temp = temp[temp.target_id.isin(hypoxia_genes.target_id)]

# should not be in the genes that go up in hif-1 mutants:
nothif1 = thomas.beta_filtered['f']
nothif1 = nothif1[(nothif1.qval < 0.1) & (nothif1.b > 0)]
nothif2 = thomas.beta_filtered['c']
nothif2 = nothif2[(nothif2.qval < 0.1) & (nothif2.b > 0)]

ind1 = ~temp.target_id.isin(nothif1.target_id)
ind2 = ~temp.target_id.isin(nothif2.target_id)
temp = temp[(ind1) & (ind2)]
temp.shape

In [ ]:
temp['b_sm'] = hypoxia_genes.b
temp['qval_avg_sm'] = hypoxia_genes.avg_qval
temp['avg_b_sm'] = hypoxia_genes.avg_b

gold = temp[temp.b*temp.b_sm > 0]

In [ ]:
ind = (genpy.find(hypoxia_genes, hypoxia_gold.WBIDS, col='ens_gene'))
found = hypoxia_genes[ind].ens_gene.unique()
sig = len(hypoxia_genes)
pval = stats.hypergeom.sf(len(found), len(thomas.beta_filtered['a']), len(hypoxia_gold), sig)    

In [ ]:
pval

In [ ]:
hypoxia_genes[ind].ext_gene.unique()

In [ ]:
len(gold)

In [ ]:
gold[gold.target_id.isin(tf_df.target_id)][['ext_gene', 'qval_avg_sm', 'avg_b_sm']]

# Hypoxia Positive Interaction

In [ ]:
# find all the genes in the hypoxia response:
ovb = thomas.beta_filtered['b'].copy()
ovd = thomas.beta_filtered['d']
ove = thomas.beta_filtered['e']
ovc = thomas.beta_filtered['c']
ova = thomas.beta_filtered['a']

ovb['d_qval'] = ovd.qval
ovb['e_qval'] = ove.qval
ovb['c_qval'] = ovc.qval
# ovb['a_qval'] = ova.qval

ovb['d_b'] = ovd.b
ovb['e_b'] = ove.b
ovb['c_b'] = ovc.b
# ovb['a_b'] = ova.b

# qvalue cutoff
ind1 = (ovb.qval < 0.1)
# ind2 = (ovb.d_qval < 0.1)
ind3 = (ovb.e_qval < 0.1)
ind4 = (ovb.c_qval < 0.1)
# ind5 = (ovb.a_qval < 0.1)

i1 = ind1 & ind2 & ind3 & ind4 #& ind5

# make sure they all go in the same direction:
# ind5 = (ovb.b*ovb.d_b > 0) & (ovb.b*ovb.e_b > 0) & (ovb.d_b*ovb.e_b >0)
# ind6 = (ovb.b*ovb.c_b > 0) & (ovb.c_b*ovb.e_b > 0) & (ovb.d_b*ovb.c_b >0)

ind5 =  (ovb.b*ovb.e_b > 0)
ind6 = (ovb.b*ovb.c_b > 0) & (ovb.c_b*ovb.e_b > 0)
# ind7 = (ovb.b*ovb.a_b > 0) & (ovb.c_b*ovb.a_b > 0) & (ovb.e_b*ovb.a_b > 0)

i2 = ind5 & ind6 #& ind7

# novel_response = ovb[ind1 & ind2 & ind3 & ind4 & ind5 & ind6].copy()
novel_response = ovb[i1 & i2].copy()


novel_response['avg_qval'] = (ovb.qval + ovb.d_qval + ovb.e_qval + ovb.c_qval)/4
novel_response['avg_b'] = (ovb.b + ovb.d_b + ovb.e_b + ovb.c_b)/4
novel_response.sort_values(['avg_qval'], inplace=True)

In [ ]:
len(novel_response.ens_gene.unique())

# Searching for a TF that is activated by both egl-9 and hif-1
And possibly regulates rhy-1 as a result

In [ ]:
# Figure out how many hits we get and if the pvalue is significant!
tfs = {}
for sm in thomas.single_mutants:
    df = thomas.beta_filtered[sm]
    ind = (genpy.find(df, tf_df.target_id, col='target_id'))
    found = df[ind & (df.qval < 0.1)]
    sig = df[df.qval < 0.1]
    pval = stats.hypergeom.sf(len(found), len(df), len(hypoxia_gold), len(sig))
    print('genotype: ', sm)
    print('found: ', len(found), '      Mean b: {0:.2g}'.format(found.b.mean()))
    print('pval: {0:.2g}'.format(pval))
    tfs[sm] = found.copy()
        
    print('----------')

In [ ]:
def add_b(x):
    vals = tfs['c'][tfs['c'].target_id == x]
    if len(vals):
        return vals.b.values[0]
    else:
        return np.nan

temp = tfs['b']
temp['b_c'] = temp.target_id.apply(add_b)

In [ ]:
temp[(temp.b_c < 0) & (temp.b < 0)][['ext_gene','b_c','b', 'qval']]

In [ ]:
# mxl-3 should be down in the egl, hif double:
temp = thomas.beta_filtered['f']
temp[(temp.ext_gene == 'mxl-3')][['ext_gene', 'b', 'qval']]

In [ ]:
temp = thomas.beta_filtered['a']
temp[(temp.ext_gene == 'mxl-3')][['ext_gene', 'b', 'qval']]

In [ ]:
temp = thomas.beta_filtered['e']
temp[(temp.ext_gene == 'mxl-3')][['ext_gene', 'b', 'qval']]

In [ ]:
temp = thomas.beta_filtered['d']
temp[(temp.ext_gene == 'mxl-3')][['ext_gene', 'b', 'qval']]

In [ ]:
# hypoxia_genes.to_csv('../hypoxia_genes.csv',index=False)
from bokeh.resources import CDN
from bokeh.embed import file_html
from IPython.core.display import HTML

# Use Bokeh chart to make plot
hypoxia_genes['logq'] = -np.log(hypoxia_genes.avg_qval)
hypoxia_genes['logq'] = hypoxia_genes['logq'].astype(np.float64)
hypoxia_genes['avg_b'].astype(np.float64)
hypoxia_genes.logq.fillna(700, inplace=True)
hypoxia_genes.logq.replace([np.inf], 700, inplace=True)
hypoxia_genes.logq.replace([-np.inf], 0, inplace=True)

hypoxia_genes.b.replace([np.inf], 6, inplace=True)
hypoxia_genes.b.replace([-np.inf], -6, inplace=True)
hypoxia_genes.dropna(inplace=True)

# What pops up on hover?
tooltips = [('ext_gene', '@ext_gene')]

# Make the hover tool
hover = bokeh.models.HoverTool(tooltips=tooltips)

# Create figure
p = bokeh.plotting.figure(plot_width=650, 
                          plot_height=450)
p.xgrid.grid_line_color = 'white'
p.ygrid.grid_line_color = 'white'
p.xaxis.axis_label ='b'
p.yaxis.axis_label ='-log(Q)'

# Add the hover tool
p.add_tools(hover)

# Define colors in a dictionary to access them with
# the key from the pandas groupby funciton.
source1 = bokeh.models.ColumnDataSource(hypoxia_genes[hypoxia_genes.avg_b < 0])
source2 = bokeh.models.ColumnDataSource(hypoxia_genes[hypoxia_genes.avg_b > 0])
# Specify data source
p.circle(x='avg_b', y='logq', size=7, alpha=0.2, source=source1, color='blue')
p.circle(x='avg_b', y='logq', size=7, alpha=0.2, source=source2, color='red')
p.legend.background_fill_alpha = 0.25
p.legend.background_fill_color = 'blanchedalmond'
html = file_html(p, CDN, "my plot")
HTML(html)

In [ ]:
bs = np.array([])
for key in thomas.beta.keys():
    df = thomas.beta_filtered[key]
    if len(bs) == 0:
        bs = df.b.values
        qs = df.qval.values
    else:
        bs = np.vstack((bs, df.b.values))
        qs = np.vstack((qs, df.qval.values))

In [ ]:
b_df = pd.DataFrame(np.transpose(bs), columns = thomas.beta.keys())
q_df = pd.DataFrame(np.transpose(qs), columns = thomas.beta.keys())
b_df = pd.melt(b_df, var_name='genotype', value_name='b')
q_df = pd.melt(q_df, var_name='genotype', value_name='q')
b_df['abs b'] = b_df.b.abs()
b_df['sorter'] = b_df.genotype.map(sorter)
q_df['sorter'] = q_df.genotype.map(sorter)
b_df.sort_values('sorter', inplace=True)
q_df.sort_values('sorter', inplace=True)
b_df['names'] = b_df.genotype.map(genotype_mapping)

In [ ]:
sns.boxplot(x="names", y="abs b", data=b_df[q_df.q < 0.1], whis=np.inf, linewidth=1)
sns.stripplot('names', 'abs b', data=b_df[q_df.q < 0.1], linewidth=1, jitter=True, alpha=0.05)
plt.xticks(rotation=45)
plt.ylim(10**-1, 10)
plt.yscale('symlog')

In [ ]:
for genotype in b_df.genotype.unique():
    print(genotype, b_df[(b_df.genotype == genotype) & (q_df.q < 10**-1)].shape[0])

# Finding direct targets of *hif-1*, *vhl-1*, *egl-9* and *rhy-1*

## *hif-1* direct targets
Knocking out *hif-1* should decrease levels of hydroxylated and non-hydroxylated *hif-1*

Knocking out *vhl-1* should increase levels of both forms. 

Knocking out *egl-9* or *rhy-1* should decrease the hydroxylated form and increase the non-hydroxylated form. 

Knocking out *egl-9;hif-1* should decrease levels of hydroxylated and non-hydroxylated *hif-1*

Therefore:
Overlap *hif-1*, *egl-9* and *rhy-1* mutants with coexpression and find only targets that go DOWN. Next, overlap *vhl-1* with anti-expression. These are the hydroxylated targets of *hif-1* and possibly some other stuff

In [ ]:
df1 = thomas.beta['f'].copy() # egl-9;hif-1 double mutant
df2 = thomas.beta['c']
df3 = thomas.beta['e']
df4 = thomas.beta['b']
df5 = thomas.beta['a'] # egl-9; vhl-1 double should also decrease hydroxylation of hif
df6 = thomas.beta['d']

df1['b_c'] = df2.b
df1['b_e'] = df3.b
df1['b_b'] = df4.b
df1['b_a'] = df5.b
df1['b_d'] = df6.b


df1['q_c'] = df2.qval
df1['q_e'] = df3.qval
df1['q_b'] = df4.qval
df1['q_a'] = df5.qval
df1['q_d'] = df6.qval

ind = (df1.qval < 0.1) & (df1.q_c < 0.1) & (df1.q_e < 0.1) &\
      (df1.q_b < 0.1) & (df1.q_a < 0.1) & (df1.q_d < 0.1)
ind2 = (df1.b < 0) & (df1.b_c < 0) & (df1.b_e < 0) & (df1.b_b < 0) & (df1.b_a < 0)
ind3 = (df1.b_d > 0)

df1[ind & ind2 & ind3][['ext_gene', 'b_d', 'q_d']]


We find two genes. . It's possible that we just couldn't measure the hydroxylated *hif-1* targets properly because 
A less stringent filter would be to just anti-overlap the *vhl-1* with the (*egl-9* and *rhy-1*) mutants and see what comes up. This means we are no longer measuring *hif-1* directly, rather, we are inferring it and hoping that the noise from other stuff that these genes have in common gets drowned out. 

In [ ]:
ind = (df1.q_e < 0.1) & (df1.q_b < 0.1) & (df1.q_a < 0.1) & (df1.q_d < 0.1) # significant in all relevant depts.
ind2 = (df1.b_e < 0) & (df1.b_b < 0) & (df1.b_a < 0) # loss of egl causes hif-OH to go down
ind3 = (df1.b_d > 0) # vhl causes hif-OH to go UP
hydroxylated_hif = df1[ind & ind2 & ind3]
hydroxylated_hif[['ext_gene', 'b_d', 'q_d', 'b_e']].sort_values('q_d').head()

Nope! It looks that even our less stringen filter yields quite the small list of targets!

Let's find the genes associated with non-hydroxylated hypoxia factor next. 

Knocking out *hif-1* should decrease levels of hydroxylated and non-hydroxylated *hif-1*

Knocking out *vhl-1* should increase levels of both forms. 

Knocking out *egl-9* or *rhy-1* should decrease the hydroxylated form and increase the non-hydroxylated form. 

Knocking out *egl-9;hif-1* should decrease levels of hydroxylated and non-hydroxylated *hif-1*

Therefore:
Overlap *vhl-1*, *egl-9* and *rhy-1* mutants with coexpression and find only targets that go UP. Next, overlap *hif-1* with anti-expression. These are  targets of non-hydroxylated *hif-1* and possibly some other stuff

In [ ]:
ind = (df1.qval < 0.1) & (df1.q_c < 0.1) & (df1.q_e < 0.1) & (df1.q_b < 0.1) & (df1.q_a < 0.1) & (df1.q_d < 0.1)
ind2 = (df1.b_e > 0) & (df1.b_b > 0) & (df1.b_a > 0) & (df1.b_d > 0)
ind3 = (df1.b < 0) & (df1.b_c < 0)
df1[ind & ind2 & ind3][['ext_gene', 'b_e', 'q_e']].sort_values('q_e')

Again, we observe only very few genes. Let's try to remove the *hif-1;egl-9* double mutant to increase our sample size:

In [ ]:
ind = (df1.q_c < 0.1) & (df1.q_e < 0.1) & (df1.q_b < 0.1) & (df1.q_a < 0.1) & (df1.q_d < 0.1)
ind2 = (df1.b_e > 0) & (df1.b_b > 0) & (df1.b_a > 0) & (df1.b_d > 0)
ind3 = (df1.b_c < 0)
df1[ind & ind2 & ind3][['ext_gene', 'b_e', 'q_e']].sort_values('q_e')

We can definitely say that these are the best candidates for direct control by *hif-1*. However, the list could probably still be larger. 

Let's weaken the conditions a little bit. Now, we will only require that whatever our candidate genes are, they should not be **significantly upregulated in response to loss of hif-1**.

In [ ]:
ind =  (df1.q_e < 0.1) & (df1.q_b < 0.1) & (df1.q_a < 0.1) & (df1.q_d < 0.1)
ind2 = (df1.b_e > 0) & (df1.b_b > 0) & (df1.b_a > 0) & (df1.b_d > 0)
ind3 = ~((df1.q_c < 0.1) & (df1.b_c > 0)) & ~((df1.qval < 0.1) & (df1.b > 0))
hypoxia_direct_targets = df1[ind & ind2 & ind3]
print(hypoxia_direct_targets.shape[0])
hypoxia_direct_targets[['ext_gene', 'b_e', 'q_e']].sort_values('q_e').head()


Now we're talking! We can definitely say that these are the best candidates for direct control by *hif-1*.

If we are willing to let go of the difference between hydroxylated and non-hydroxylated *hif-1*, we could remove the *hif-1* filters entirely. However, I think these 167 genes are probably enough for now. Let's take a moment now to verify that this result matches what is known in the literature:

In [ ]:
_ = tea.enrichment_analysis(hypoxia_direct_targets.ens_gene.unique(), tissue_df, show=True)
_ = tea.enrichment_analysis(hypoxia_direct_targets.ens_gene.unique(), phenotype_df, show=True)

## Quality control on identified genes:

Before we call these *hif-1* targets, we should make sure that at least some known targets are contained in this set. In order to do this, we have curated a list of 20 or so genes that have been published before as *hif-1* targets.

That being said though, we need to be aware of 1 major issue with this dataset. 

*hif-1* and *rhy-1* form an incredibly tight loop. There is a LOT of feedback between *hif-1* and *rhy-1*. Given the kind of logic we are using, we are probably excluding a number of targets as a result. In fact, the logic we have used to develop this list excludes *rhy-1* itself, a known *hif-1* target! I could do better, but not without a lot more lines of code, and it just doesn't seem reasonable to do this. 

In [ ]:
ind = hypoxia_direct_targets.ens_gene.isin(hypoxia_gold.WBIDS)
found = hypoxia_direct_targets[ind].ens_gene.unique()
sig = len(hypoxia_direct_targets)
pval = stats.hypergeom.sf(len(found), len(thomas.beta_filtered['a']), len(hypoxia_gold), sig)    

if pval < 10**-3:
    print('This result is statistically significant \
with a p-value of {0:.2g} using a hypergeometric test. You found {1} gold standard genes!'.format(pval, len(found)))

Very nice! We are actually sampling from the hif-1 pool! Fantastic! 

## Identifying *rhy-1* targets

Next, let's identify rhy-1 associated genes. We will also insist that rhy-1 genes NOT be associated with hif-1 (they really shouldn't be, the logic between these two sets is mutually exclusive, I think, but it's best to make sure; let's hit stuff with a hammer):

In [ ]:
ind =  (df1.q_e < 0.1) & (df1.q_b < 0.1) & (df1.q_a < 0.1) & (df1.q_c < 0.1)
ind2 = (df1.b_e*df1.b_b > 0) & (df1.b_e*df1.b_a > 0) & (df1.b_c*df1.b_e > 0) & (df1.b_c*df1.b_b > 0)
ind3 = (~df1.target_id.isin(hypoxia_direct_targets.target_id))
rhy1_targets = df1[ind & ind2 & ind3]
print(rhy1_targets.shape)
rhy1_targets[['ext_gene', 'b_e', 'q_e']].sort_values('q_e').head()

In [ ]:
_ = tea.enrichment_analysis(rhy1_targets.ens_gene.unique(), tissue_df, show=True)
_ = tea.enrichment_analysis(rhy1_targets.ens_gene.unique(), phenotype_df, show=False)
_[0].head()

In [ ]:
rhy1_targets[(rhy1_targets.b_e <= rhy1_targets.b_e.min() + 1.5)][['ext_gene','b_e']]

## Identifying *egl-9* targets
OK! We found 'em. Let's find the *egl-9* related genes next.

In [ ]:
ind =  (df1.q_e < 0.1) & (df1.q_b < 0.1) & (df1.q_a < 0.1) & (df1.q_d > 0.1) & (df1.q_c > 0.1)
ind2 = (df1.b_e*df1.b_b > 0) & (df1.b_e*df1.b_a > 0) & (df1.b_b*df1.b_a > 0)
#         & (df1.b_c*df1.b_e < 0) & (df1.b_c*df1.b_b < 0)\
#         & (df1.b_b*df1.b_d > 0) & (df1.b_e*df1.b_d > 0)
ind3 = True
ind4 = (~df1.target_id.isin(hypoxia_direct_targets.target_id))
egl_targets = df1[ind & ind2 & ind3  & ind4]
print(egl_targets.ens_gene.unique().shape[0])
egl_targets[['ext_gene', 'b_b', 'q_b']].sort_values('q_b').head(10)

In [ ]:
_ = tea.enrichment_analysis(egl_targets.ens_gene.unique(), tissue_df, show=True)
_ = tea.enrichment_analysis(egl_targets.ens_gene.unique(), phenotype_df, show=False)
_[0].head(10)

## Identifying *vhl-1* targets
Next, let's find the vhl-related genes.

In [ ]:
ind =  (df1.q_a < 0.1) & (df1.q_d < 0.1)
ind2 = (df1.b_d > 0) & (df1.b_a > 0)
ind4 = (df1.q_e > 0.1) & (df1.q_b > 0.1) & (df1.q_b > 0.1)

vhl_targets = df1[ind & ind2 & ind4]
print(vhl_targets.shape[0])
vhl_targets[['ext_gene', 'b_d', 'q_d', 'b_b', 'b_e']].sort_values('q_d').head(10)

In [ ]:
_ = tea.enrichment_analysis(vhl_targets.ens_gene.unique(), tissue_df, show=True)
_ = tea.enrichment_analysis(vhl_targets.ens_gene.unique(), phenotype_df, show=False)
_[0].head(10)

## Identifying New Biology - understanding the role of *rhy-1* and *egl-9* in the *hif-1* dependent response



In [ ]:
ind = (df1.q_c < 0.1) & (df1.q_e < 0.1) &\
      (df1.q_b < 0.1) & (df1.q_a < 0.1) & (df1.q_d < 0.1)
y = df1[ind][['b_e', 'b_b', 'b_d', 'b_c', 'b', 'b_a']]
y.shape

In [ ]:
all_down = y[y<0].dropna().index
all_down.shape

In [ ]:
all_down = df1[df1.index.isin(y[y<0].index)][['ext_gene', 'ens_gene', 'b_e', 'b_b', 'b_d', 'b_c', 'b_a',
                               'q_e', 'q_b',
                               'q_d', 'q_c', 'q_a']].sort_values('q_a')
all_down.to_csv('../output/all_down.csv', index=False)

In [ ]:
all_up = y[y>0].dropna().index
print(all_up.shape)
all_up = df1[df1.index.isin(all_up)][['ext_gene', 'ens_gene', 'b_e', 'b_b', 'b_d', 'b_c', 'b_a',
                               'q_e', 'q_b',
                               'q_d', 'q_c', 'q_a']].sort_values('q_a')
all_up.to_csv('../output/all_up.csv', index=False)

In [ ]:
ind = (df1.q_c < 0.1) & (df1.q_e < 0.1) &\
      (df1.q_b < 0.1) & (df1.q_d < 0.1)
y = df1[ind][['b_e', 'b_b', 'b_d', 'b_c']]
y.shape

In [ ]:
ind = (df1.q_c < 0.1) & (df1.q_e < 0.1) &\
      (df1.q_b < 0.1) & (df1.q_d < 0.1) & (df1.qval < 0.1)
y = df1[ind][['b_e', 'b_b', 'b_d', 'b_c']]
y.shape

In [ ]:
plt.plot(y.b_e, y.b_a, 'ro')
plt.plot(y.b_d, y.b_a, 'bo')
x = np.linspace(plt.gca().get_xlim()[0], plt.gca().get_xlim()[1], plt.gca().get_xlim()[1] - plt.gca().get_xlim()[0])
plt.plot(x, x)

In [ ]:
plt.plot(y.b_e, y.b_a - y.b_e, 'ro')
plt.plot(y.b_d, y.b_a - y.b_d, 'bo')

In [ ]:
plt.plot(y.b_c, y.b - y.b_c, 'ro')
plt.plot(y.b_e, y.b - y.b_e, 'bo')

In [ ]:
plt.plot(y.b_c, y.b, 'ro')
plt.plot(y.b_e, y.b, 'bo')